In [15]:
import os
import re
from pdf2image import convert_from_path 
from PIL import Image

from PyPDF2 import PdfFileMerger, PdfFileReader, PdfFileWriter
import pathlib
# 获取当前目录
curr_dir = pathlib.Path('.')

In [5]:
dir_input = input()
curr_dir = pathlib.Path(dir_input)

 /Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg


In [14]:
def pdf_to_img(pdf_path):
    pages = convert_from_path(pdf_path)
    image_counter = 1
    for page in pages: 
        page.save(f'page{image_counter}.jpg', 'JPEG')
        image_counter = image_counter + 1

In [18]:

def pdf_to_img(pdf_path):
    pages = convert_from_path(pdf_path)
    image_counter = 1
    for page in pages: 
        page.save(f'page{image_counter}.jpg', 'JPEG')
        image_counter = image_counter + 1

In [88]:

def pdf_to_img(pdf_path):
    pages = convert_from_path(pdf_path)
    image_counter = 1
    for page in pages: 
        page.save(f'page{image_counter}.jpg', 'JPEG')
        image_counter = image_counter + 1


def img_split(path):
    img = Image.open(path)
    width, height = img.size
    
    # 计算切割后的片段数量
    split_width = width
    split_num_w = 1
    split_height = width * 1.4
    split_num_h = int(height // split_height)
    
    # 获取图片主色调
    main_color = get_main_color(img)
    
    # 开始切割
    i = 0
    sum_offset = 0 #总偏移 初始化
    offset = 5
    for j in range(split_num_h+1):
        # 计算切割区域 sum_offset 上次总偏移
        x1, x2 = 0, split_width
        y1, y2 = j*split_height + sum_offset + int(j>0)*offset, min((j+1)*split_height, img.height)
        area = (x1, y1, x2, y2)

        # 检查切割线附近的颜色,判断是否过于接近文本
        # if is_nearlywhite(img, area, main_color, 5):  
        #     continue   # 跳过切割
        sum_offset = 0 #总偏移清零
        while not is_nearlywhite(img, area, main_color, offset):
            sum_offset += offset
            y2 = min((j+1)*split_height + sum_offset, img.height)
            area = (x1, y1, x2, y2)
            if y2 == img.height:
                break
        if y1 >= y2:
            continue
        crop_img = img.crop(area)  
        # 保存切割后的片段
        # todo 
        path_img = curr_dir/'img/'
        print(path_img/f'{os.path.splitext(path)[0]}_{i}_{j:02d}.jpg')
        crop_img.save(path_img/f'{os.path.splitext(path)[0]}_{i}_{j:02d}.jpg') 

# 获取图片主色调            
def get_main_color(img):
    # 将图片转换为RGB模式
    img = img.convert('RGB')
    
    # 获取图片尺寸
    width, height = img.size
    
    # 统计图片中RGB值出现的次数
    rgb_dict = {}
    for i in range(width):
        for j in range(height):
            rgb = img.getpixel((i,j))
            if rgb not in rgb_dict:
                rgb_dict[rgb] = 1
            else:
                rgb_dict[rgb] += 1
                
    # 将字典按value值排序,取出现次数最多的颜色        
    rgb_sorted = sorted(rgb_dict.items(), key=lambda x:x[1], reverse=True)
    
    # 返回RGB值最高的颜色            
    return rgb_sorted[0][0]  

def get_linearea_4point(img, area, offset):
    x1, y1, x2, y2 = area
    # 扩大offset个像素
    x1_o = x1 - offset
    y1_o = y2 - offset
    x2_o = x2 + offset
    y2_o = y2 + offset

    # 校验扩大后的区域是否超出图像范围
    x1_o = max(x1_o, 0)
    y1_o = int(min(max(y1_o, 0), img.height))
    x2_o = int(min(x2_o, img.width))
    y2_o = int(min(y2_o, img.height))
    return (x1_o, y1_o, x2_o, y2_o)


# 检查指定区域附近是否为白色或背景色        
def is_nearlywhite(img, area, main_color, offset):
    # 获取切割区域左上角和右下角位置
    x1, y1, x2, y2 = area
    (x1_o, y1_o, x2_o, y2_o) = get_linearea_4point(img, area, offset)
    if y1_o >= y2_o:
        return True
    # 统计扩大区域内每个像素的值
    rgb_dict = {}
    for i in range(x1_o, x2_o):
        for j in range(y1_o, y2_o):
            rgb = img.getpixel((i,j))
            if rgb not in rgb_dict:
                rgb_dict[rgb] = 1
            else:
                rgb_dict[rgb] += 1  
                
    # 如果主色调的像素值大于80% 或 最近的其他颜色值不超过10%
    if rgb_dict[main_color] > 0.95 * (x2_o - x1_o) * (y2_o - y1_o) \
            or rgb_dict[max(rgb_dict, key=rgb_dict.get)] < 0.05 * (x2_o - x1_o) * (y2_o - y1_o):
        return True
    else:
        return False


In [20]:
pdf_path = "00test.pdf"
pdf_to_img(pdf_path)
path = "."


PDFPageCountError: Unable to get page count.
I/O Error: Couldn't open file '00test.pdf': No such file or directory.


In [6]:

tmp = pathlib.Path('.')
# tmp = curr_dir
path_img = curr_dir/'img/'

In [90]:
path_img

PosixPath('/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg/img')

In [86]:
# 递归遍历所有jpg文件
for filename in curr_dir.glob('*.jpg'):
    print(filename)
    # img = Image.open(filename)
    img_split(filename)

/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg/0.发刊词  为什么你知道这么多，还是睡不好？.jpg
/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg/3.减少阻力：创造合适的卧室环境.jpg
/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg/4.排除干扰：酒、咖啡、安眠药和午觉 .jpg


In [42]:
curr_dir

PosixPath('/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg')

In [73]:
filename

PosixPath('0.发刊词  为什么你知道这么多，还是睡不好？_0_09.jpg')

In [79]:
path = pathlib.Path('/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg')
# path.mkdir()
i = 0
j = 1
path_img = path/'img/'
# path_img.mkdir()
print(path_img/f'{os.path.splitext(filename)[0]}_{i}_{j:02d}.jpg')

f'{os.path.splitext(filename)[0]}_{i}_{j:02d}.jpg'

/Users/kk/Downloads/048 怎样获得高质量睡眠（不整理）jpg/img/0.发刊词  为什么你知道这么多，还是睡不好？_0_09_0_01.jpg


'0.发刊词  为什么你知道这么多，还是睡不好？_0_09_0_01.jpg'

# image2pdf

In [102]:

# import pathlib
# # 获取所有文件
# files = os.listdir(path_img)
# os.chdir(path_img)
# # 对文件名排序 todo
# files.sort()

# # 使用lambda表达式获取文件名中的两段数字部分 
# # files.sort(key=lambda f: (int(f.split('_')[2]), int(f.split('_')[3])))

# # 初始化PdfFileMerger对象
# merger = PdfFileMerger()

# # 遍历图片并添加到PDF
# for filename in files:
#     if not filename.endswith('.jpg'):
#         continue
#     print(filename)
#     # 打开图片
#     image = Image.open(filename)

#     # 添加空白页并将图片添加到PDF
#     page = PdfFileReader(open(filename, 'rb')).getPage(0)
#     page.scaleTo(image.width, image.height)

#     # 使用%格式化
#     merger.append(PdfFileReader('%s' % filename, 'rb').getPage(0))  

#     # 为指定图片添加书签
#     if filename.endswith('_0_00.jpg'):
#         bk_name = (filename[0:len(filename)-len('_0_00.jpg')])
#         bkmk = merger.addBookmark(bk_name, len(merger.pages) - 1) 

# # 输出PDF文件
# # merger.write('result.pdf') 

0.发刊词  为什么你知道这么多，还是睡不好？_0_00.jpg


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [11]:
from PIL import Image 
import img2pdf
from PyPDF2 import PdfFileMerger
from io import BytesIO

import pathlib

files = os.listdir(path_img)
os.chdir(path_img)

# curr_dir = pathlib.Path('.')
# # 获取所有文件
# files = os.listdir(curr_dir)

# 对文件名排序 
files.sort()
# 使用lambda表达式获取文件名中的数字部分
# files.sort(key=lambda f: int(f.split('_')[2])) 


# 初始化PdfFileMerger对象
merger = PdfFileMerger()

# 遍历图片并添加到PDF
for img in files:
    if not img.endswith('.jpg'):
        continue
    # 打开图片
    # print(img)
    image = Image.open(img)  

    # 图片转换为PDF页面
    page = img2pdf.convert(img)  

    # 添加页面
    f = BytesIO(page) 
    merger.append(f)

    # 为指定图片添加书签
    if img.endswith('_0_00.jpg'):
        bk_name = (img[0:len(img)-len('_0_00.jpg')])
        bkmk = merger.addBookmark(bk_name, len(merger.pages) - 1) 
        print(bk_name)


# 输出PDF文件 
merger.write('result.pdf')

0.发刊词  为什么你知道这么多，还是睡不好？
1.增大动力：睡眠就像滑滑梯
2.减少阻力：创造合适的心理环境
3.减少阻力：创造合适的卧室环境
4.排除干扰：酒、咖啡、安眠药和午觉 
5.形成习惯：改善睡眠微习惯 


In [3]:

# import pathlib
# from io import BytesIO
# # 获取所有文件
# os.chdir('.')
# files = os.listdir('.')
# # 对文件名排序 todo
# files.sort()

# # 使用lambda表达式获取文件名中的两段数字部分 
# # files.sort(key=lambda f: (int(f.split('_')[2]), int(f.split('_')[3])))

# # 初始化PdfFileMerger对象
# merger = PdfFileMerger()

# # 图片转换为PDF页面  
# page = img2pdf.convert(img)

# # 构造文件对象并传入  
# f = BytesIO(page)  
# merger.append(f)
# # 遍历图片并添加到PDF
# for filename in files:
#     if not filename.endswith('.jpg'):
#         continue
#     print(filename)
#     # 打开图片
#     image = Image.open(filename)

#     # 添加空白页并将图片添加到PDF
#     page = PdfFileReader(open(filename, 'rb')).getPage(0)
#     page.scaleTo(image.width, image.height)

#     # 使用%格式化
#     merger.append(PdfFileReader('%s' % filename, 'rb').getPage(0))  

#     # 为指定图片添加书签
#     if filename.endswith('_0_00.jpg'):
#         bk_name = (filename[0:len(filename)-len('_0_00.jpg')])
#         bkmk = merger.addBookmark(bk_name, len(merger.pages) - 1) 

# # 输出PDF文件
# # merger.write('result.pdf') 

0.发刊词  为什么_0_00 .jpg


UnicodeDecodeError: 'utf-8' codec can't decode byte 0xff in position 0: invalid start byte

In [52]:
for f in files:
    if f.startswith('page1_'):
        print(f, [int(n) for n in re.findall(r'\d+', f)] )

page1_0_10.jpg [1, 0, 10]
page1_0_9.jpg [1, 0, 9]
page1_0_8.jpg [1, 0, 8]
page1_0_0.jpg [1, 0, 0]
page1_0_1.jpg [1, 0, 1]
page1_0_3.jpg [1, 0, 3]
page1_0_2.jpg [1, 0, 2]
page1_0_6.jpg [1, 0, 6]
page1_0_7.jpg [1, 0, 7]
page1_0_5.jpg [1, 0, 5]
page1_0_4.jpg [1, 0, 4]


In [95]:
import re 
import os
from collections import defaultdict

files = os.listdir(curr_dir) 
files = [f for f in files  if f.startswith('page1_')]
print(files)
nums = [[int(n) for n in re.findall(r'\d+', f)]
        for f in files if f.startswith('page1_')]
print(nums)
f='page1_0_10.jpg'
print(nums[files.index(f)])

files.sort(key=lambda f: nums[files.index(f)], reverse=True) 

files.sort(key=lambda f: nums[files.index(f)]
          )   

files.sort(key=lambda f: (nums[files.index(f)][0]
                          , nums[files.index(f)][1]
                          , nums[files.index(f)][2])
          )   
              
# 对文件名排序  
files.sort(key=lambda f: (int(f.split('_')[2]), int(f.split('_')[3])))   

print(files)
for f in files:
    print(f)

['page1_0_10.jpg', 'page1_0_9.jpg', 'page1_0_8.jpg', 'page1_0_0.jpg', 'page1_0_1.jpg', 'page1_0_3.jpg', 'page1_0_2.jpg', 'page1_0_6.jpg', 'page1_0_7.jpg', 'page1_0_5.jpg', 'page1_0_4.jpg']
[[1, 0, 10], [1, 0, 9], [1, 0, 8], [1, 0, 0], [1, 0, 1], [1, 0, 3], [1, 0, 2], [1, 0, 6], [1, 0, 7], [1, 0, 5], [1, 0, 4]]
[1, 0, 10]


ValueError: 'page1_0_10.jpg' is not in list

## 迁移书签

In [ ]:
import PyPDF2

files = os.listdir(path_img)
os.chdir(path_img)

# 读取源PDF文件并获取书签信息
pdf = PyPDF2.PdfFileReader('source.pdf')
bookmarks = pdf.getOutlines()

# 遍历书签,获取位置和名称
for bookmark in bookmarks:
    page_num = bookmark.page
    title = bookmark.title

# 读取目标PDF文件,准备添加书签  
pdf_writer = PyPDF2.PdfFileWriter()
pdf_writer.cloneReaderDocumentRoot(PyPDF2.PdfFileReader('target.pdf'))

# 添加书签
bookmark_page = pdf_writer.addBookmark(title, page_num-1)   

# 输出新的PDF  
with open('new.pdf', 'wb') as f:
    pdf_writer.write(f) 